In [ ]:
import pyfits,glob,time,scipy,re,time
import scipy.interpolate
from numpy import *
import numpy
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from subprocess import call
import ipyparallel
from itertools import cycle
from scipy import misc
from scipy.ndimage import shift

matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15)
matplotlib.rc('font',size=30)

import pylab
pylab.rcParams['font.size']=30
topdir='/Users/cslage/Research/LSST/code/GUI/'

%matplotlib inline

In [ ]:
def mosaic_combine_ITL(filename):
    "Takes in a multi-extension mosaic fits file and the dimensions of the mosaic and returns an array with all chips combined"
    dim_x,dim_y=8,2
    hdr0 = pyfits.getheader(filename,0)
    numccds = hdr0['nextend']
    sizex,sizey = hdr0['IMG_COLS'],hdr0['IMG_ROWS']
    overscanx,overscany = hdr0['OVR_COLS'],hdr0['OVR_ROWS']
    prescanx,prescany = hdr0['PRE_COLS'],hdr0['PRE_ROWS']
    sizex,sizey = sizex,sizey
    
    ccdarray = np.zeros((sizey*dim_y,sizex*dim_x),dtype='float')
        
    for ccdnum in range(numccds):
        ccdseg = pyfits.getdata(filename,ccdnum+1)
        med_overscan=median(ccdseg[:,-overscanx:])
        ccdseg=ccdseg[prescany:-overscany,prescanx:-overscanx]-med_overscan
        j=ccdnum/dim_x   # depends on int division
        if ccdnum<dim_x:
            i=dim_x-ccdnum-1
            ccdseg=ccdseg[:,::-1]
        else:
            i=ccdnum-dim_x
            ccdseg=ccdseg[::-1,:]
        ccdarray[j*sizey:(j+1)*sizey,i*sizex:(i+1)*sizex]=ccdseg
    ccdbottom=ccdarray[0:sizey*dim_y/2,0:sizex*dim_x]
    ccdarray[0:sizey*dim_y/2,0:sizex*dim_x]=ccdbottom[:,::-1]
    return ccdarray

def make_whisker_array2(cat,ndiv,r_out,sigma_smooth,):
    """Takes in a sextractor catalog of shapes and makes a whisker plot. Needs the catalog (in LDAC format),
    the number of divisions to cut the image into and the maximum distance to consider a galaxy to smooth
    inside a division, and the smoothing fwhm in pixels"""
    xs,ys=4072,4000
    xg=cat.field('XWIN_IMAGE')
    yg=cat.field('YWIN_IMAGE')
    thetag=cat.field('THETA_IMAGE')
    ag=cat.field('A_IMAGE')
    bg=cat.field('B_IMAGE')

    r=bg/ag
    eg = (1.-r)/(1.+r)
    #de=np.sqrt(x2err**2+y2err**2)
    #e1,e2 = e*np.cos(2.*theta),e*np.sin(2.*theta)
    
    #First size up the field, and then break it up into pieces for individual analysis
    dx = xs/ndiv
    dy = ys/ndiv
    
    #initialize some arrays to hold the subset calculations
    xc,yc = numpy.zeros([ndiv,ndiv]),numpy.zeros([ndiv,ndiv])
    angsub,arrow_len = numpy.zeros([ndiv,ndiv]),numpy.zeros([ndiv,ndiv])
    ngal = numpy.zeros([ndiv,ndiv])
    agrid= numpy.zeros([ndiv,ndiv])
    bgrid= numpy.zeros([ndiv,ndiv])
    
    #go division by division
    for i in range(ndiv):
        for j in range(ndiv):
            # find the center of the division
            xc[i,j] = ((i*1./ndiv)*xs + dx/2)
            yc[i,j] = ((j*1./ndiv)*ys + dy/2)
            #grab the good elements that are inside the subdivision
            sub = numpy.where((abs(xg-xc[i,j])<r_out) & (abs(yg-yc[i,j])<r_out))
            ngal[i,j] = len(sub[0])   #the number of galaxies used in the calc
            #calculate each element's distance from the center
            dx2 = (xc[i,j]-xg[sub])**2
            dy2 = (yc[i,j]-yg[sub])**2
            r2 = dx2 + dy2
            #and then apply the radius from center in a gaussian weight
            weight = 1.*numpy.exp(-.5*r2/sigma_smooth**2)#/degsub**2
            weightsum = numpy.sum(weight)
            weight=weight/weightsum
            agrid[i,j] = numpy.sum(ag[sub]*weight)
            bgrid[i,j] = numpy.sum(bg[sub]*weight)
            scalar_ellip = numpy.sum(eg[sub]*weight)#numpy.sqrt(e1sub[i,j]**2+e2sub[i,j]**2)
            arrow_len[i,j] = scalar_ellip*xs
            angsub[i,j] = numpy.sum(thetag[sub]*weight)
            #phi = .5*numpy.arctan2(e2sub[i,j],e1sub[i,j])
    return xc,yc,arrow_len,angsub,ngal,agrid,bgrid

def run_sextractor_on_mosaic(fitsfilename):
    #from mydefs import make_ell_reg_from_xy
    #from mydefs import make_reg_from_ldac
    import pyfits,numpy
    from subprocess import call

    configfile=topdir+'sextractor/30-micron-pinholes.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'

    #configfile='/home/andrew/Work/ccd/30-micron-pinhole.sex'
    #paramfile='/home/andrew/Work/ccd/default-array_dither.param'
    hdr0=pyfits.open(fitsfilename)[0].header
    pdiode=hdr0['MONDIODE']
    exptime=hdr0['EXPTIME']
    fluxval=float(pdiode)*float(exptime)
    minadu=5e10*fluxval
    outname=fitsfilename
    callstring=["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat',"-CATALOG_TYPE","FITS_LDAC",
               "-CHECKIMAGE_NAME",outname[:-4]+"back.fits"+','+outname[:-4]+"rms.fits",
               "-CHECKIMAGE_TYPE","BACKGROUND,BACKGROUND_RMS",
               "-DETECT_THRESH","10","-DETECT_MINAREA","10","-THRESH_TYPE","RELATIVE","-DETECT_MAXAREA","400",
               "-ANALYSIS_THRESH","2"]
    test=call(callstring)
    #make_ell_reg_from_xy(fitsfilename)
    #make_ell_reg_from_xy(outname)
    #call(["rm",outname])
    #print callstring



def filtcat_mediansig(cat,parname,nsig):
    """Filters a sextractor catalog python object by sigma clipping parname in cat by signum,
    returns the max,min and std deviation of vals"""
    parvals=cat[parname]
    parvals=parvals#[where(cat['FLAGS']==0)]
    medparval=median(parvals)
    stdparval=std(parvals)
    maxparval,minparval=medparval+nsig*stdparval,medparval-nsig*stdparval
    parvals=parvals[where((parvals<maxparval) & (parvals>minparval))]
    medparval=median(parvals)
    stdparval=std(parvals)
    maxparval,minparval=medparval+nsig*stdparval,medparval-nsig*stdparval
    goodobjs=where((cat[parname]<maxparval) & (cat[parname]>minparval))# & (cat['FLAGS']==0))
    return cat[goodobjs],minparval,maxparval,stdparval



In [ ]:
def make_stick_plot114042(arrow_len,angsub,ngal,ev,pct_scale):
    """Makes a stick plot for the 8x2 sensor, taking the array ( ndiv x ndiv ) of the arrow lengths, angles,
    and number of galaxies in each division. Also take a parameter which says what fraction of sticks to plot
    and what the scale should be (generally .01->1.0"""
    xs,ys=4072,4000
    ngal_min=max(ngal.flatten())*.2
    figure(figsize=(10,10))
    axis([-200,xs+200,-200,ys+200])
    for i in range(8):
        plt.arrow(509*i,0,0,4000,lw=2,alpha=.5)
    arrow(0,2000,4072,0,lw=2,alpha=.5)
    arrow(0,0,4072,0,lw=2,alpha=.5)
    arrow(4072,0,0,4000,lw=2,alpha=.5)
    arrow(0,4000,4072,0,lw=2,alpha=.5)
    #pcolormesh(xc,yc,ngal,alpha=.1)#,cmap=cm.Blues)
    #colorbar()
    #create the vectors to be plotted
    U = arrow_len*numpy.cos(angsub)
    V =arrow_len*numpy.sin(angsub)
        
    M=numpy.zeros([ndiv,ndiv])
    M[numpy.where(ngal<ngal_min)] = True
    U = arrow_len*numpy.cos(angsub*pi/180.)
    V = arrow_len*numpy.sin(angsub*pi/180.)
    maskedU = numpy.ma.masked_array(U,mask=M)
    maskedV = numpy.ma.masked_array(V,mask=M)
    #use the quiver function in pylab to plot our vectors (easier than 'arrow')
    # only plot every (ev) X number of arrows to reduce clutter (ev defined at top)
    q = plt.quiver(xc[::ev,::ev],yc[::ev,::ev],maskedU[::ev,::ev],maskedV[::ev,::ev],
                   headaxislength=0,units='x',headlength=0,width=4,pivot='mid',alpha=1,scale=pct_scale)
    #make a key for the quivers, with length of 10% ellipticity
    #qk = quiverkey(q,xs/2,ys/2.,.1*xs,'1% ellipticity',coordinates='data')
    plt.axis('off')

In [ ]:
def make_ell_reg_from_xy(filename):
    """Takes in X & Y coordinates and a text tag and outputs a DS9 region file"""
    foocat=pyfits.getdata(filename+'.cat',"LDAC_OBJECTS")
    xreg,yreg=foocat['XWIN_IMAGE'],foocat['YWIN_IMAGE']
    radius1=7*np.ones(len(xreg))
    radius2=foocat['B_IMAGE']/foocat['A_IMAGE']*radius1
    anglereg=foocat['THETA_IMAGE']
    text_tag=['']*len(xreg)
    reg_filename=filename+'.reg'
    f = open(reg_filename,'w')
    f.write('image'+' \n')
    for i in range(len(xreg)):
        thetext=text_tag[i]
        f.write('ellipse '+str(xreg[i])+' '+str(yreg[i])+' '+str(radius1[i])+' '+str(radius2[i])+
                ' '+str(anglereg[i])+' #text="'+str(thetext)+'"\n')
    f.close()

In [ ]:
thedir='/Users/cslage/Research/LSST/code/GUI/cte/20170404-bf-30um-V1/'
expnum_root,date='2','20170404'
filelist=sort(glob.glob(thedir+'ITL-3800C-029_spot_spot_'+expnum_root+'??_'+date+'??????_ct.fits'))
print len(filelist)," files"
zpos_all=[]
for filename in filelist:
    zpos=pyfits.getheader(filename,17)['Y_POS']
    zpos_all.append(zpos)
    print '#',filename.split('/')[-1].split('_')[3],zpos

In [ ]:
tstart=time.time()
for filename in filelist[:]:
    pyfits.writeto(filename[:-5]+'whole.fits',mosaic_combine_ITL(filename),header=pyfits.getheader(filename,0),clobber='True')

print time.time()-tstart

In [ ]:
mosfilelist=sort(glob.glob(thedir+'ITL-3800C-029_spot_spot_'+expnum_root+'??_'+date+'??????_ctwhole.fits'))
print len(mosfilelist)

In [ ]:
tstart=time.time()
for i in range(len(mosfilelist)): run_sextractor_on_mosaic(mosfilelist[i])

print time.time()-tstart

In [ ]:
# Now you should be able to check out the fits images and region files together.
# to open the first one (which is usually a good check) open the .fits and .reg like:
# ds9 *_800_*whole.fits -regions *_800_*.reg

In [ ]:
ncatobjs=[]
for fname in mosfilelist:
    ncatobjs.append(len(pyfits.getdata(fname+'.cat',"LDAC_OBJECTS")))
figure(figsize=(20,5))
print ncatobjs
plot(ncatobjs)
xticks(linspace(0,len(mosfilelist),41))
xlabel('exposure #')

In [ ]:
filenum=20
mosfilename=mosfilelist[filenum]
print mosfilename
catname=mosfilelist[filenum]+'.cat'
cat=pyfits.getdata(catname,"LDAC_OBJECTS")
print str(len(cat))+' total objects before filtering'
parnames=['A_IMAGE','B_IMAGE','X2WIN_IMAGE','Y2WIN_IMAGE','FLUX_RADIUS','MAG_AUTO','MAGERR_AUTO','FLUX_APER','FLUXERR_APER']
nsig=3
for parname in parnames:
    cat,minparval,maxparval,stdparval=filtcat_mediansig(cat,parname,nsig)

print str(len(cat))+' after'

# plot up the measured sextractor parameters for the filtered catalog
nbins=100
figure(figsize=(15,15))
subplot(221)
title('Ellipse parameters')
hist(cat['A_IMAGE'],bins=nbins,histtype='step',label='A_IMAGE',normed='True',lw=3)
hist(cat['B_IMAGE'],bins=nbins,histtype='step',label='B_IMAGE',normed='True',lw=3)
hist(cat['FLUX_RADIUS'],bins=nbins,histtype='step',label='FLUX_RADIUS',normed='True',lw=3)
xlabel('Sizes [pixels]')
ylabel('N')
legend()
subplot(222)
title('Second moments')
hist(cat['X2WIN_IMAGE'],bins=nbins,histtype='step',label='X2_IMAGE',lw=3)
hist(cat['Y2WIN_IMAGE'],bins=nbins,histtype='step',label='Y2_IMAGE',lw=3)
hist(cat['XYWIN_IMAGE'],bins=nbins,histtype='step',label='XY_IMAGE',lw=3)
print cat['FLUX_MAX']
legend()
subplot(223)
title('Magnitudes and errors')
plot(cat['MAG_AUTO'],cat['MAGERR_AUTO'],'k.')
xlabel('MAG_AUTO')
ylabel('MAGERR_AUTO')
subplot(224)
title('Orientation angles')
hist(cat['THETA_IMAGE'],bins=250,histtype='step',lw=3)
xlabel('THETA_IMAGE')
for i in range(-4,4,1):
    axvline(90./4.*i,color='k',linestyle='--',linewidth=1)#"""

savefig(thedir+"Moments1.png")

In [ ]:
import itertools

def polyfit2d(x, y, z, order=3):
    ncols = (order + 1)**2
    G = np.zeros((x.size, ncols))
    ij = itertools.product(range(order+1), range(order+1))
    for k, (i,j) in enumerate(ij):
        G[:,k] = x**i * y**j
    coeff, residues, rank, singval = np.linalg.lstsq(G, z)
    return coeff,residues,rank,singval

def polyval2d(x, y, m):
    order = int(np.sqrt(len(m))) - 1
    ij = itertools.product(range(order+1), range(order+1))
    z = np.zeros_like(x)
    for a, (i,j) in zip(m, ij):
        z += a * x**i * y**j
    return z
def get_stamps(xc,yc,imagename,halfstampsize):
    img=pyfits.getdata(imagename)
    sz=halfstampsize
    nobjs=len(xc)
    stampstack=np.zeros((nobjs,sz*2,sz*2))
    maxvals=np.zeros(nobjs)
    x_sz_max,y_sz_max=3900,3900
    for ind in range(nobjs):
        x,y=xc[ind]+.5,yc[ind]+.5
        if ((x<sz) | (x >x_sz_max-sz-1) | (y<sz) | (y>y_sz_max-sz-1)): 
            continue
        xf,yf=np.floor(x),np.floor(y)
        onestamp=img[int(y-sz):int(y+sz),int(x-sz):int(x+sz)]
        stampstack[ind,:,:]=onestamp/np.max(onestamp)
        maxvals[ind]=np.max(onestamp)
    return stampstack,maxvals

#sys.path.insert(1,'/home/andrew/Work/ccd/wavefronts/')
#from donutengine import donutengine
from psf_evaluator import Moment_Evaluator
ME=Moment_Evaluator()

In [ ]:
#xmin,xmax,ymin,ymax=1000,2000,1500,2500
#xmin,xmax,ymin,ymax=500,3500,500,3500
xmin,xmax,ymin,ymax=100,3900,100,3900
thenum=19
halfstampsize=8
cat=pyfits.getdata(mosfilelist[thenum]+'.cat',"LDAC_OBJECTS")
print mosfilelist[thenum]
obj_inds=np.where((cat['XWIN_IMAGE']<xmax) & (cat['YWIN_IMAGE']<ymax) & (cat['MAG_AUTO']<-1) & (cat['MAGERR_AUTO']<.003) &
           (cat['XWIN_IMAGE']>xmin) & (cat['YWIN_IMAGE']>ymin) )[0]
print len(obj_inds)
xc,yc=cat['XWIN_IMAGE'][obj_inds],cat['YWIN_IMAGE'][obj_inds]
x2sex,y2sex=cat['X2WIN_IMAGE'][obj_inds],cat['Y2WIN_IMAGE'][obj_inds]

# grab stamps at the locations of 
stamps,maxvals=get_stamps(xc,yc,mosfilelist[thenum],halfstampsize)

In [ ]:
plt.plot(cat['MAG_AUTO'],cat['MAGERR_AUTO'],'ko',label='all')
plt.plot(cat['MAG_AUTO'][obj_inds],cat['MAGERR_AUTO'][obj_inds],'go',label='used objects')
plt.legend()
plt.yscale('log')
plt.xlabel('MAG_AUTO'),plt.ylabel('MAGERR_AUTO')

In [ ]:
x2mod,y2mod=[],[]
x2obs,y2obs=[],[]
x3obs,y3obs=[],[]
tstart=time.time()
for i in range(len(obj_inds)):
    stamp=stamps[i]
    stampmoms=ME(stamp)
    x2obs.append(stampmoms['x2'])
    y2obs.append(stampmoms['y2'])
    x3obs.append(stampmoms['x3'])
    y3obs.append(stampmoms['x3'])
x2mod,y2mod=np.array(x2mod),np.array(y2mod)
x2obs,y2obs=np.array(x2obs),np.array(y2obs)
x3obs,y3obs=np.array(x3obs),np.array(y3obs)

print time.time()-tstart

In [ ]:
plt.figure(figsize=(14,7))
plt.subplot(121)
ev=10
#plt.plot(xc[::ev],x2mod,'k.',alpha=.7,label='x2 mod')
plt.plot(xc[::ev],x2obs[::ev],'b.',alpha=.7,label='x2 obs')
plt.plot(xc[::ev],x2sex[::ev],'r.',alpha=.7,label='x2 sex')
plt.legend(loc='upper right')
plt.ylim(.5,.9)
plt.xlabel('X COORD'),plt.ylabel('X MOMENT')

plt.subplot(122)
#plt.plot(yc[::ev],y2mod,'k.',alpha=.7,label='y2 mod')
plt.plot(yc[::ev],y2obs[::ev],'b.',alpha=.7,label='y2 obs')
plt.plot(yc[::ev],y2sex[::ev],'r.',alpha=.7,label='y2 sex')
plt.legend(loc='upper right')
plt.ylim(.5,.9)
plt.xlabel('Y COORD'),plt.ylabel('Y MOMENT')

savefig(thedir+"Moments2.png")

In [ ]:
ev=1
plt.plot(xc,x2obs[::ev]-x2sex[::ev],'k.',alpha=.05)
plt.plot(xc,y2obs[::ev]-y2sex[::ev],'r.',alpha=.05)
plt.xlabel('X COORD'),plt.ylabel('MOMENT DIFFERENCES ($ADAPTIVE - SEX$)')
plt.ylim(-.1,.1)

In [ ]:
#keys=['07','06','05','04','03','02','00','17','16','15','14','13','12','11','10']#,'10']#,'07','16','01','00'

keys=['06','05','04','02','00','16','15','14','13','12','11','10']

ctidict={'07':2.5e-5,'06':1.4e-4,'05':1.4e-4,'04':2.7e-5,'03':1.84e-4,'02':1.0e-5,'01':6.5e-6,'00':1.5e-5,
          '17':1.6e-5,'16':9.1e-6,'15':6.3e-6,'14':6.3e-6,'13':7.7e-6,'12':7.5e-6,'11':8.8e-6,'10':1.6e-5}
 
ctis=np.array([ctidict[key] for key in keys])


segxmins={'07':0,'06':509,'05':509*2,'04':509*3,'03':509*4,'02':509*5,'01':509*6,'00':509*7,
          '17':0,'16':509,'15':509*2,'14':509*3,'13':509*4,'12':509*5,'11':509*6,'10':509*7}
segymins={'07':2000,'06':2000,'05':2000,'04':2000,'03':2000,'02':2000,'01':2000,'00':2000,
          '17':1000,'16':1000,'15':1000,'14':1000,'13':1000,'12':1000,'11':1000,'10':1000}



x2tofit,y2tofit=x2sex,y2sex#x2obs,y2obs#
polyorder=2
nx, ny = 100,100
ginds=np.where((np.isnan(x2tofit)==False) & (np.isnan(y2tofit)==False) & ((np.abs(yc-1000)>1000) | (np.abs(xc-1250)>350))& (x2tofit < 0.75) & (y2tofit < 0.75))
xx, yy = np.meshgrid(np.linspace(0, 4000, nx), 
                     np.linspace(0, 4000, ny))
plt.figure(figsize=(12,8))
x2poly,res,rank,sval = polyfit2d(xc[ginds],yc[ginds],x2tofit[ginds],order=polyorder)
x2z = polyval2d(xx, yy, x2poly)
y2poly,res,rank,sval = polyfit2d(xc[ginds],yc[ginds],y2tofit[ginds],order=polyorder)
y2z = polyval2d(xx, yy, y2poly)
plt.xlabel('X COORD'),plt.ylabel('Y COORD')

ev=3
x2omin,x2omax,y2omin,y2omax=.5,1.5,.5,1.5
plt.contourf(xx,yy,x2z,levels=np.linspace(x2omin,x2omax,100),cmap=plt.cm.RdBu_r)
plt.scatter(xc[ginds][::ev],yc[ginds][::ev],c=x2tofit[ginds][::ev],vmin=x2omin,vmax=x2omax,cmap=plt.cm.RdBu_r),plt.colorbar()
plt.title('X MOMENT')
for key in keys: plt.text(segxmins[key],segymins[key],key,fontsize=20,color='w')


plt.figure(figsize=(12,8))
plt.title('Y MOMENT')
for key in keys: plt.text(segxmins[key],segymins[key],key,fontsize=20,color='w')
plt.contourf(xx,yy,y2z,cmap=plt.cm.RdBu_r,levels=np.linspace(y2omin,y2omax,100))
plt.scatter(xc[ginds][::ev],yc[ginds][::ev],c=y2tofit[ginds][::ev],cmap=plt.cm.RdBu_r,vmin=y2omin,vmax=y2omax),plt.colorbar()
plt.xlabel('X COORD'),plt.ylabel('Y COORD')


In [ ]:
plt.figure(figsize=(14,7))
plt.subplot(121)
ev=10
x2z = polyval2d(xc, yc, x2poly)
plt.plot(xc[::ev],x2z[::ev],'b.',alpha=.7,label='fit')
plt.plot(xc[::ev],x2sex[::ev],'r.',alpha=.7,label='x2 sex')
plt.legend(loc='upper right')
plt.ylim(.5,.9)
plt.xlabel('X COORD'),plt.ylabel('X MOMENT')

plt.subplot(122)
y2z = polyval2d(xc, yc, y2poly)
plt.plot(yc[::ev],y2z[::ev],'b.',alpha=.7,label='fit')
plt.plot(yc[::ev],y2sex[::ev],'r.',alpha=.7,label='y2 sex')
plt.legend(loc='upper right')
plt.ylim(.5,.9)
plt.xlabel('Y COORD'),plt.ylabel('Y MOMENT')

savefig(thedir+"Moments_Fit.png")

In [ ]:
plt.hist(x2tofit-x2z,range=[-.05,.05])

In [ ]:
import seaborn as sb

In [ ]:
evfoo=1
vmin,vmax=-.03,.03
x2z = polyval2d(xc, yc, x2poly)

#plt.contourf(xc,yc,x2z,levels=np.linspace(x2omin,x2omax,100),cmap=plt.cm.RdBu_r)
plt.figure(figsize=(12,8))
plt.title('X MOMENT RESIDUALS')
plt.scatter(xc[::evfoo],yc[::evfoo],c=(x2tofit-x2z)[::evfoo],vmin=vmin,vmax=vmax,cmap=plt.cm.RdBu_r,marker='o',alpha=.3),plt.colorbar()
plt.axis([0,4e3,0,4e3])
for key in keys: plt.text(segxmins[key],segymins[key],key,fontsize=20,color='w')
savefig(thedir+"XResiduals_Trunc.png")
plt.figure(figsize=(12,8))
plt.title('Y MOMENT RESIDUALS')
y2z = polyval2d(xc, yc, y2poly)
#plt.contourf(xc,yc,y2z,cmap=plt.cm.RdBu_r,levels=np.linspace(y2omin,y2omax,100))
plt.scatter(xc[::evfoo],yc[::evfoo],c=(y2tofit-y2z)[::evfoo],vmin=vmin,vmax=vmax,cmap=plt.cm.RdBu_r,marker='o',alpha=.3),plt.colorbar()
plt.axis([0,4e3,0,4e3])
for key in keys: plt.text(segxmins[key],segymins[key],key,fontsize=20,color='w')
savefig(thedir+"YResiduals_Trunc.png")

In [ ]:
nbins=100
plt.figure(figsize=(18,10))
x2resid,y2resid=[],[]
for key in keys:
    x2fix,y2fix=[],[]
    xmin,ymin,cti=segxmins[key],segymins[key],7e-8
    #print(key,xmin,ymin)
    gseg=np.where((xc>xmin) & (xc<xmin+509) & (yc>ymin) & (yc<ymin+1000))[0]
   
    for g in gseg:
        x2fix.append(polyval2d(xc[g], yc[g], x2poly)-x2tofit[g])
        y2fix.append(polyval2d(xc[g], yc[g], y2poly)-y2tofit[g])
    x2fix,y2fix=np.array(x2fix),np.array(y2fix)
    x2resid.append(np.median(x2fix))
    y2resid.append(np.median(y2fix))
    plt.subplot(211)
    plt.hist(x2fix,bins=nbins,histtype='step',label=key,range=[-.05,.05],lw=2)
    plt.text(np.median(x2fix),90,key)#,markersize=12)#,label='SEGMENT'+key
    plt.axvline(np.median(x2fix),color='k',linestyle='--')
    plt.subplot(212)
    plt.hist(y2fix,bins=nbins,histtype='step',label=key,range=[-.05,.05],lw=2)
    plt.text(np.median(y2fix),90,key)#,markersize=12)#,label='SEGMENT'+key
    plt.axvline(np.median(y2fix),color='k',linestyle='--')
    print key,len(gseg),np.median(x2fix),np.median(y2fix)
plt.xlabel('Y MOMENT RESIDUALS')
plt.subplot(211)
plt.xlabel('X MOMENT RESIDUALS')
x2resid,y2resid=np.array(x2resid),np.array(y2resid)
plt.legend(ncol=2)
savefig(thedir+"ResidualHist_Trunc.png")

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(121)
for key,i in zip(keys,range(len(keys))):
    plt.subplot(121)
    plt.plot(x2resid[i],ctis[i],marker='$'+str(key)+'$',markersize=12)#,label='SEGMENT'+key
    #plt.plot(x2resid_adap[i],ctis[i],marker='$'+str(key)+'$',markersize=12)#,label='SEGMENT'+key
    #plt.text(float(x2resid[i]),float(ctis[i]),str(key))
    plt.subplot(122)
    plt.plot(y2resid[i],ctis[i],marker='$'+str(key)+'$',markersize=12)#,label='SEGMENT'+key)
    #plt.plot(y2resid_adap[i],ctis[i],marker='$'+str(key)+'$',markersize=12)#,label='SEGMENT'+key)
plt.subplot(121)
#plt.legend(loc='upper left')
plt.yscale('log')
plt.xlabel('residual X moment')
plt.ylabel('$CTI_{EPER}$')

plt.subplot(122)
#plt.plot(y2resid,ctis,'ro')
plt.yscale('log')
plt.xlabel('residual Y moment')
plt.ylabel('$CTI_{EPER}$')
savefig(thedir+"Residuals_vs_CTI_Trunc.png")

In [ ]:
ndiv=64
r_out=200
sigma_smooth=150
xc,yc,arrow_len,angsub,ngal,agrid,bgrid=make_whisker_array2(cat,ndiv,r_out,sigma_smooth)

In [ ]:
figure(figsize=(8,8))
title('Ellipticity magnitude for exposure #: '+str(filenum),fontsize=20)
hist((arrow_len/4000).flatten(),bins=100,histtype='step')
xlabel('ellipticity, $e=\\frac{1-r}{1+r}$',fontsize=25)
ylabel('Number of pinholes',fontsize=25)

In [ ]:
ev=2
pct_scale=1#.05
make_stick_plot114042(arrow_len,angsub,ngal,ev,pct_scale)
title("Exp #:"+mosfilename.split('/')[-1].split('_')[3]+', z pos='+str(zpos_all[filenum])+', # of obj: '+str(len(cat)),fontsize=20)

In [ ]:
plt.hist(cat['XWIN_IMAGE']-np.floor(cat['XWIN_IMAGE']),bins=100,histtype='step')
xlabel("subpixel centroid")

In [ ]:

figure(figsize=(8,5))
for fname in mosfilelist[:1]:
    hdr0=pyfits.open(fname)[0].header
    pdiode=float(hdr0['MONDIODE'])
    exptime=float(hdr0['EXPTIME'])
    fluxval=pdiode*exptime

    minadu=2e11*fluxval
    catname=fname+".cat"
    cat=pyfits.getdata(catname,"LDAC_OBJECTS")
    fmax=cat['FLUX_MAX']
    xc,yc=cat['XWIN_IMAGE'],cat['YWIN_IMAGE']
    g=where((fmax>minadu) & (cat['FLAGS']==0) & 
            ((cat['XWIN_IMAGE']-np.floor(cat['XWIN_IMAGE'])<.1) | (cat['XWIN_IMAGE']-np.floor(cat['XWIN_IMAGE'])>.9)))# &
            #(cat['YWIN_IMAGE']-np.floor(cat['YWIN_IMAGE'])>.4) & (cat['XWIN_IMAGE']-np.floor(cat['XWIN_IMAGE'])<.6) &)[0]
    plot(cat['XWIN_IMAGE'][g],np.sqrt(cat['X2WIN_IMAGE'][g]),'k.',alpha=.1)
for i in range(1,8): axvline(509*i,color='k',linestyle='--')
#axis([0,4e3,.4,.8])
xlabel('X centroid')
ylabel('X second moment')

In [ ]:
theta_all=[]
for mosfilename in mosfilelist:
    theta_all.extend(pyfits.getdata(mosfilename+'.cat',"LDAC_OBJECTS")['THETA_IMAGE'][:])
theta_all=array(theta_all)

In [ ]:
print len(theta_all)

In [ ]:
figure(figsize=(12,6))
hist(theta_all,bins=10000,color='b',histtype='step',range=[-90,89.9])
xlabel('THETA_IMAGE',fontsize=20)
ylabel('Number of pinholes')
title('Angular distribution of '+str(len(theta_all))+' pinholes')
for i in range(-6,7):
    axvline(i*180/12,linestyle='--',color='k')

# making stacked images

In [ ]:
def make_stamp_stack_subpix(cat,imagename,obj_inds,halfstampsize,nzoom):
    sz=halfstampsize
    nobjs=len(obj_inds)
    img=pyfits.getdata(imagename)
    xc,yc=cat['XWIN_IMAGE'],cat['YWIN_IMAGE']
    nstamps=0
    stampstack=np.zeros((sz*nzoom*2,sz*nzoom*2))
    xsize,ysize=3900,3900
    for k in range(nobjs):
        ind=obj_inds[k]
        x,y=xc[ind]+.5,yc[ind]+.5
        if ((x<sz) | (x >xsize-sz-1) | (y<sz) | (y>ysize-sz-1)): 
            continue
        xf,yf=np.floor(x),np.floor(y)
        xshift,yshift=round((x-xf)*(nzoom)),round((y-yf)*(nzoom))
        onestamp=img[y-sz:y+sz,x-sz:x+sz]
        onestamp=onestamp/np.max(onestamp)
        regridstamp=np.zeros((sz*nzoom*2,sz*nzoom*2))
        for i in range(sz*2):
            for j in range(sz*2):
                i0,i1=nzoom*i,nzoom*(i+1)
                j0,j1=nzoom*j,nzoom*(j+1)
                regridstamp[i0:i1,j0:j1]=onestamp[i,j]
        shiftstamp=shift(regridstamp,(-yshift+nzoom,-xshift+nzoom),order=0)
        stampstack+=shiftstamp
        nstamps+=1

    stampstack=stampstack/nstamps
    return stampstack

def make_mosaic_stamp_subpix(stackfitsname,halfwin,ndiv,nzoom):
    """Takes in a fits file and searches for its catalog, then stacks object images by shifting and medianing. 
    Halfwin is the half size of the window around each object, and ndiv is how many divisions to split the image into"""
    sexcat=pyfits.getdata(stackfitsname+'.cat',"LDAC_OBJECTS")
    fw=(halfwin*2*nzoom)
    xmossize,ymossize=4071,3999#4072,4000
    xgridsz,ygridsz=xmossize/ndiv*1.,ymossize/ndiv*1.
    xgridmins,ygridmins=linspace(0,xmossize,ndiv),linspace(0,ymossize,ndiv)
    allstampstacks=np.zeros(((halfwin*2*nzoom+1)*ndiv,(halfwin*2*nzoom+1)*ndiv))
    for i in range(ndiv):
        xgridmin,xgridmax=i*xgridsz,(i+1)*xgridsz
        for j in range(ndiv):
            ygridmin,ygridmax=j*ygridsz,(j+1)*ygridsz
            g=where((sexcat['XWIN_IMAGE']>xgridmin) & (sexcat['XWIN_IMAGE'] < xgridmax) &
                    (sexcat['YWIN_IMAGE']>ygridmin) & (sexcat['YWIN_IMAGE'] < ygridmax) &
                    (sexcat['FLAGS']==0))[0]
            allstampstacks[j*fw:(j+1)*fw,i*fw:(i+1)*fw]=make_stamp_stack_subpix(sexcat,stackfitsname,g,halfwin,nzoom)
    return allstampstacks

In [ ]:
thenum=10 # frame number to analyze
nzoom=20  #how much to subdivide a pixel
halfstampsize=3  #the half width of the postage stamp to get
xmin,xmax=1500,2500   #usually a good idea to not use all the pinholes near the edges
ymin,ymax=1000,3000

for thenum in [0,19]:
    cat=pyfits.getdata(mosfilelist[thenum]+'.cat',"LDAC_OBJECTS")
    obj_inds=where((cat['XWIN_IMAGE']<xmax) & (cat['YWIN_IMAGE']<ymax) &
               (cat['XWIN_IMAGE']>xmin) & (cat['YWIN_IMAGE']>ymin) & (cat['FLAGS']==0))[0]
    stampstack=make_stamp_stack_subpix(cat,mosfilelist[thenum],obj_inds,halfstampsize,nzoom)
    
    figure(figsize=(5,5))
    pcolor(stampstack)#,colorbar()
    for i in range(2*halfstampsize+1):
        axvline(i*nzoom,color='w')
        axhline(i*nzoom,color='w')
    axis('off')

In [ ]:
# you can also make a mosaic of these for each region of the CCD
# to make a "compressed & enhanced" view of the image
# (this is especially useful for visualizing aberrations in focus curves)

In [ ]:
filenum=14
stackfitsname=mosfilelist[filenum]

halfwin,ndiv,nzoom=4,8,10
tstart=time.time()
allstampstacks=make_mosaic_stamp_subpix(stackfitsname,halfwin,ndiv,nzoom)
print time.time()-tstart

In [ ]:
figure(figsize=(15,15))
from matplotlib.colors import LogNorm  #optional lognorm flag below
imshow(allstampstacks,origin='lower')#,norm=LogNorm(.01,1.1),cmap='rainbow')
axis('off')

In [ ]:
### some definitions to calculate your own weighted image moments

In [ ]:
def calc_firstmoms(img,coord,winsize,thresh,gwin_sig):
    "Calculates the x, y, xy, and radial second moments of the subimg, given the image and its centroid" 
    xc,yc=coord
    xmom,ymom=0.,0.
    thesum=0.
    
    i0=int(round(xc)-winsize)
    j0=int(round(yc)-winsize)
    #print i0,j0,winsize
    for i in range(2*winsize+1):
        for j in range(2*winsize+1):
            if img[j0+j,i0+i]>thresh:
                gwt=np.exp(-((xc-(i0+i))**2+(yc-(j0+j))**2)/(2.*gwin_sig**2))
                thesum+=img[j0+j,i0+i]*gwt
                xmom+=img[j0+j,i0+i]*((i0+i))*gwt
                ymom+=img[j0+j,i0+i]*((j0+j))*gwt
    if thesum >0:
        xmom= xmom/thesum
        ymom= ymom/thesum
    else:
        rmom=xmom=ymom=xymom=1.
    return xmom, ymom

def calc_secmoms_gwin(img,coord,winsize,thresh,gwin_sig):
    "Calculates the x, y, xy, and radial second moments of the subimg, given the image and its centroid" 
    xc,yc=coord
    r2mom,x2mom,y2mom,xy2mom=0.,0.,0.,0.
    thesum=0.
    
    i0=int(round(xc)-winsize)
    j0=int(round(yc)-winsize)
    #print i0,j0,winsize
    for i in range(2*winsize+1):
        for j in range(2*winsize+1):
            if img[j0+j,i0+i]>thresh:
                gwt=np.exp(-((xc-(i0+i))**2+(yc-(j0+j))**2)/(2.*gwin_sig**2))
                thesum+=img[j0+j,i0+i]*gwt
                r2mom+=img[j0+j,i0+i]*((xc-(i0+i))**2+(yc-(j0+j))**2)*gwt
                x2mom+=img[j0+j,i0+i]*(xc-(i0+i))**2*gwt
                y2mom+=img[j0+j,i0+i]*(yc-(j0+j))**2*gwt
                xy2mom+=img[j0+j,i0+i]*(((i0+i)-xc)*((j0+j)-yc))*gwt
    if thesum >0:
        rmom = np.sqrt(.5*(r2mom/thesum))
        xmom= np.sqrt(x2mom/thesum)
        ymom= np.sqrt(y2mom/thesum)
        xyz=xy2mom/thesum
        xymom= xyz/abs(xyz)*np.sqrt(abs(xyz))
    else:
        rmom=xmom=ymom=xymom=1.
    return rmom, xmom, ymom, xymom

def median_xy(xin,yin,nxbins):
    xmin,xmax=xin.min(),xin.max()
    xbin=np.linspace(xmin,xmax,nxbins+1)
    ymed=np.zeros(nxbins)
    num_objs=np.zeros(nxbins)
    for i in range(nxbins):
        gd=np.where((xin>xbin[i]) & (xin<xbin[i+1]))
        ymed[i]=np.median(yin[gd])
        num_objs[i]=len(gd[0])
    return np.array(xbin[:-1]),ymed,num_objs

In [ ]:
np.shape(stampstack)